# Chinese Topic Modeling — Harry Potter Fanfiction

In these lessons, we're learning about a text analysis method called *topic modeling*. This method will help us identify the main topics or discourses within a collection of texts a single text that has been separated into smaller text chunks.

---

## Dataset

### Harry Potter Fanfiction (Chinese)

In this particular lesson, we're going to use [Little MALLET Wrapper](https://github.com/maria-antoniak/little-mallet-wrapper), a Python wrapper for [MALLET](http://mallet.cs.umass.edu/topics.php), to topic model a CSV file with fanfiction stories from the website [Archive of Our Own (AO3)](https://archiveofourown.org/).

___

<div class="admonition attention" name="html-admonition" style="background: lightyellow;padding: 10px">  
<p class="title">Attention</p>  
    
<p>If you're working in this Jupyter notebook on your own computer, you'll need to have both the Java Development Kit and MALLET pre-installed. For set up instructions, please see <a href="http://melaniewalsh.github.io/Intro-Cultural-Analytics/Text-Analysis/Topic-Modeling-Set-Up.html">the previous lesson<a/>.  </p>
    
If you're working in this Jupyter notebook in the cloud via Binder/JupyterHub, then the Java Development Kit and Mallet will already be installed. You're good to go!  
     
</div>  

## Set MALLET Path

Since Little MALLET Wrapper is a Python package built around MALLET, we first need to tell it where the bigger, Java-based MALLET lives.

We're going to make a variable called `path_to_mallet` and assign it the file path of our MALLET program. We need to point it, specifically, to the "mallet" file inside the "bin" folder inside the "mallet-2.0.8" folder. 

In [ ]:
path_to_mallet = '../mallet/bin/mallet'

If MALLET is located in another directory, then set your `path_to_mallet` to that file path.

## Install Packages

In [ ]:
#!pip install little_mallet_wrapper
#!pip install seaborn
#To install the most updated version of little_mallet_wrapper:
#!!pip install git+https://github.com/maria-antoniak/little-mallet-wrapper.git

## Import Packages

In [ ]:
import little_mallet_wrapper
import seaborn
import pandas as pd
import random
from pathlib import Path
pd.options.display.max_colwidth = 100

## Get Training Data From CSV File

In [ ]:
hp_df_chinese = pd.read_csv("harry_potter_chinese_stories_first_chaps.csv")

In [ ]:
hp_df_chinese

Drop rows with no story text

In [ ]:
hp_df_chinese = hp_df_chinese.dropna(subset=['Text'])

## Text Pre-Processing

To topic model Chinese texts, we're going to segment the texts and then remove Chinese stop words. 

In [ ]:
!python -m spacy download zh_core_web_sm

In [ ]:
import zh_core_web_sm
import spacy
nlp = spacy.load("zh_core_web_sm")

Run the spacy nlp model on each text

In [ ]:
nlp_documents = list(nlp.pipe(hp_df_chinese[:200]['Text'].to_list()))

Pull out the tokens (segement the text)

In [ ]:
all_tokens = []
for doc in nlp_documents:
    doc_tokens = []
    for token in doc:
        token_text = token.text
        doc_tokens.append(token_text)
    doc_tokens = " ".join(doc_tokens)    
    all_tokens.append(doc_tokens)

Load Chinese stopwords so we can remove them. This is the source that we're using: https://github.com/stopwords-iso/stopwords-zh It's important to closely inspect the stopwords that you choose!

In [ ]:
chinese_stop_words = open("chinese_stop_words.txt").read().split()

`little_mallet_wrapper.process_string(text, numbers='remove')`

Next we're going to process our texts with the function `little_mallet_wrapper.process_string()`. This function will take every individual post, transform all the text to lowercase as well as remove stopwords, punctuation, and numbers, and then add the processed text to our master list `training_data`.

In [ ]:
training_data = [little_mallet_wrapper.process_string(text, numbers='remove', remove_punctuation=False, remove_short_words=False, stop_words_extra = chinese_stop_words) for text in all_lemmas]

Strip punctuation from Chinese characters (strip anything that's not a word)

In [ ]:
training_data = [re.sub('\W+',' ', text) for text in training_data]

## Get original texts

In [ ]:
original_texts = [text for text in hp_df_chinese['Text'][:200]]

### Get Story Titles

We're also going to extract the file name for each Reddit post.

In [ ]:
hp_chinese_titles = [title for title in hp_df_chinese['Title'][:200]]

### Get Dataset Statistics

We can get training data summary statisitcs by using the funciton `little_mallet_wrapper.print_dataset_stats()`.

In [ ]:
little_mallet_wrapper.print_dataset_stats(training_data)

## Training the Topic Model

We need to make a variable `num_topics` and assign it the number of topics we want returned. Then we're going to set a file path where we want all our MALLET topic modeling data to be dumped.

In [160]:
# your choice of topics
num_topics = 50

path_to_mallet = '../mallet/bin/mallet'

#No need to change anything below here
training_data = training_data

#Set output directory
output_directory_path = 'topic-model-output/hp-fanfiction/chinese-firstchaps'

#Create output directory
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

#Create output files
path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"

### Train Topic Model

Then we're going to train our topic model with `little_mallet_wrapper.quick_train_topic_model()`.

In [ ]:
#little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
#                                             output_directory_path,
#                                             num_topics,
#                                             training_data)

When the topic model finishes, it will output your results to your `output_directory_path`.

## Display Topics and Top Words

To examine the topics that the topic model extracted from the Reddit posts, run the cell below. This code uses the `little_mallet_wrapper.load_topic_keys()` function to read and process the MALLET topic model output.

In [42]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

✨Topic 0✨

['dumbledore', 'grindelwald', 'albus', 'professor', '霍格沃兹', '魔法部', 'gellert', '预言家', '黑魔法', 'ale', '学生们', '戈德里克', '德姆斯特朗', 'ariana', '孩子们', '办公室', '越来越', '盖勒特格林德沃', 'aberforth', 'ariana会']

✨Topic 1✨

['omega', 'alpha', '信息素', 'beta', '生殖腔', '发情期', '抑制剂', '湿漉漉', 'alpha们', '成omega', '满脑子', '张开嘴', '悄无声息', '有一搭没一搭', 'omega们', '大幅度', '无意识', '硬邦邦', '虎视眈眈', '黑发omega']

✨Topic 2✨

['马尔福', '迷情剂', '医疗翼', '庞弗雷', '哈利猛', '斯莱特', '哈利能', '什么的', '庞弗雷夫人', '哈利皱', '一大堆', '心跳声', '好半天', '对不起', '金发斯莱特林', '不对劲', '见怪不怪', '哈利看', '哈利站', '哈利早']

✨Topic 3✨

['harry', 'draco', 'malfoy', 'hermione', 'potter', 'ron', 'sly', 'therin', 'ginny', 'pansy', 'harrypotter', 'kreacher', 'weasley', 'gryffindor', '一如既往', 'merlin', '前列腺', '迫不及待', '大部分', 'black']

✨Topic 4✨

['伏地魔', '黑魔王', '邓布利多', '毫无疑问', '凤凰社', '食死徒', '小精灵', '继承人', '食死徒们', '比如说', '孤儿院', '难以置信', '阿瓦达', '钻心咒', '争霸赛', '索命咒', '追随者', '魔法界', '攻击性', '大难不死']

✨Topic 5✨

['哈里安', '争霸赛', '克莱尔', '图书馆', '魔法部', '大部分', '打招呼', '从今以后', '格林格拉斯', '雅各布', '受不了', '黑魔法', '

## Load Topic Distributions

MALLET also calculates the likely mixture of these topics for every single document in the corpus. This mixture is really a probability distribution, that is, the probability that each topic exists in the document. We can use these probability distributions to examine which of the above topics are strongly associated with which specific documents.

To get the topic distributions, we're going to use the `little_mallet_wrapper.load_topic_distributions()` function, which will read and process the MALLET topic model output.

In [ ]:
topic_distributions = little_mallet_wrapper.load_topic_distributions(path_to_topic_distributions)

In [ ]:
topic_distributions[0]

## Display Top Documents Per Topic

In [ ]:
from IPython.display import Markdown, display
import re

def make_md(string):
    """A function that transforms string data into Markdown
    so it can be nicely formatted with bolding and emojis
    """
    display(Markdown(str(string)))

def get_top_docs(docs, topic_distributions, topic_index=1, n=5, doc_length = 2000):
    
    """A function that shows the top documents for a given set of topic distributions
    and a specific topic number
    """
    
    sorted_data = sorted([(_distribution[topic_index], _document) for _distribution, _document in zip(topic_distributions, docs)], reverse=True)
    topic_words = topics[topic_index]
    make_md(f"### ✨Topic {topic_index}✨\n\n{topic_words}\n\n---")
    
    for probability, doc in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_words:
            if word in doc.lower():
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        make_md(f'✨  \n**Topic Probability**: {probability}  \n**Document**: {doc[:doc_length]}\n\n')

# Voldemort

In [40]:
get_top_docs(original_texts, topic_distributions, topic_index=4, n=5)

### ✨Topic 4✨

['伏地魔', '黑魔王', '邓布利多', '毫无疑问', '凤凰社', '食死徒', '小精灵', '继承人', '食死徒们', '比如说', '孤儿院', '难以置信', '阿瓦达', '钻心咒', '争霸赛', '索命咒', '追随者', '魔法界', '攻击性', '大难不死']

---

✨  
**Topic Probability**: 0.5806620054725243  
**Document**: 伏地魔在三十七岁生日前夜，收到了一份意料之外的礼物。

彼时他刚结束世界环游返回英国，他曾竭尽所能去往各地学习魔法，满腹知识。现在，他比过去更为强大，将自己的归国之路畅想成一位即将君临的征服者，好比返回罗马的凯撒。

但是，在他重振雄图、再次召集食死徒前，他得先卸货整理。毕竟，当着一群仆从狗腿的面掏出某些当属人们心目中最为危险的黑魔法造物是相当不合适，他们不配得到这份荣耀……也不配得到他的信任。

于是在这里，被破败织物和厚得同栽绒羊毛没两样的蜘蛛网包围其中，他可一点也不像凯撒了，但这个古老的宅邸遗骸必须立即拾掇完毕。尽管伏地魔恶名远扬，他本人却深谙凑合将就之道，早年在孤儿院的生活教会他许多。他将于此整理自己的所有物，修生养息，为之后的计划做准备，接着就会在最有利的时机将自己的回归公之于众。他并不着急，他的时间多得很。

毕竟，他是永生不死的。

但是，就在伏地魔解开设在自己所有物上的缩小咒之际，一股疾风猛烈地穿过整座里德尔府，带来一个不知打哪冒出来的男孩。

男孩跌在灰扑扑的地板上，发出一声憋闷的惊呼。“好痛！”

这不可能。伏地魔已经在周围设下防卫，除他以外无人能过，最为强大的血缘魔法将所有人隔绝在外，除非是他的血亲。可问题在于，他没有任何血亲。他把他们都杀了。

唯有伏地魔才能够幻影移形到这里，连门钥匙也不行。仅凭空间转移是无法穿梭进来的……但时间可以。伏地魔在这股突如其来的风中骤然感受到那一掠而过的魔力影子，它属于某种最为古老的魔法。古老，却不稳定。

伏地魔不是傻子，在男孩能动弹之前已然魔杖在手，变出绳索把男孩绑了个结实。织就这张魔法束缚的是蛇——活生生的真蛇，扭曲盘绕——本该使男孩尖叫着退却，但对方只是大喊道：“见鬼，我日，滚开！”

然后那群蛇就滚了。

伏地魔瞪着他。

瞪着他。

随后，缓慢地，在极度震惊中意识到，他刚才听到了蛇佬腔。

他注视的时间越长，这个闯入者的外貌就越发清晰；他有一头乱蓬蓬的黑发，和伏地魔的很像，却更凌乱，以及对方的眼神真的分外狠戾。裹挟怒火的眼睛一对上伏地魔的，立刻别开去，但这足以揭露出那双眼眸后藏有一颗防卫得滴水不漏的心。伏地魔无声的摄魂取念如油般划过男孩内心的边缘，找不到着力点。效果削半的物理束缚把这个闯入者的身体禁锢住，可他的精神形态缥缈无形，宛如藏在一堵雾墙之后。

“你是？”伏地魔礼貌地问道。

“你最糟糕的梦魇。”那男孩厉声说，听上去近乎歇斯底里，“我的意思是，你对我来说就是如此，所以这很公平。”

“告诉我你的名字。”伏地魔逼问，往自己的话语中加入一记夺魂咒，但他毫不惊讶地发现男孩并未屈服。拥有这种水准的大脑封闭术高手并不会受此影响。

“哈利。”但男孩还是低声说道，一脸茫然。“你说什么？我的名字……?”这回轮到他透过庄园昏暗的光线注视伏地魔了。然而这种暗色下，只有伏地魔用他蛇类般的视力才看得清。在男孩眼里，伏地魔与其说是个人类模样，不如说是团阴影，所以他善解人意地为这个年轻人点亮一道温和的照明咒，解答他的困惑。哈利立刻脸色煞白：“哦梅林，你不会是——你不会是——”

“我不会是什么？”

男孩摇摇头。“算了。”

“你想知道现在是哪一年吗？”

哈利抬起头瞥了他一眼，十分警觉。

“我只能推断出，你是因为时间魔法出了意外。时间穿越的跨度超过几小时就并不可行，无论你的本事有多瞩目，我都对它能否帮你推翻魔法本身的法则充满怀疑，除非魔法自告奋勇让你得偿所愿。”他一边密切观察哈利的反应，一边说道，“现在是1963年。”

哈利的身体摇摆不定，好像被打了一拳。他的眼睛睁得很大，吓坏了。“我操。”他轻声说，相当无礼。

伏地魔的鼻翼因厌恶而翕动。所以那些本应灌输进脑子里完美无瑕的纯血礼仪，他的继承人半点没学到。不过没关系，伏地魔会教导他的，显然未来的他在这方面一败涂地。

这个“哈利”是他的儿子，再为明显不过了。

一切都顺理成章起来。现在这里有个男孩，具备卓越超凡的魔法天赋，并且和伏地魔自己的魔法特质呈现出离奇的相似度——就好像他体内怀揣了伏地魔的一部分——再者，他还是个蛇佬腔。伏地魔拜访刚特家时已经清剿了所有剩余的蛇佬腔，所以，未来第二个蛇语使用者存在的唯一理由便是伏地魔自己的子嗣。更不用提男孩面对摄魂取念时刀枪不入的内心；伏地魔只遇到过一位年纪轻轻就展露出在大脑封闭术方面极高天赋的人，而那就是他自己。

虽然伏地魔一贯倾向于保持自身的永生不朽，但毫无疑问的是，在未来的某天，他会做出拥有继承人的决定，来进一步保护自己遗产的安全——多出的这具躯壳和他自己在魔法上如此相似，若情势所需，他可以永久性地将其占为己有。这是个替代魂器的解决方法，而且操作起来更加简单可行。确实，如果继承人选择忤逆他，也算是个潜在威胁，但伏地魔什么时候拒绝过挑战呢？更不消提，见证他的子嗣是否继承到自己的强大



✨  
**Topic Probability**: 0.5678913694995141  
**Document**: 伏地魔赢了。
邓布利多死了，死于和伏地魔的决斗。老人的身体已经无法支撑如此高强度的战斗，早在决斗之前，凤凰社的人就已经有了不详的预感。黑色的诅咒锁住了老校长的生命，那只干枯的手无时不刻都在提醒他们邓布利多力量的消亡。
但他们别无选择。
决斗前，邓布利多和伏地魔立下誓言。誓言的内容无人知晓，直到邓布利多的葬礼结束，伏地魔带领食死徒走进凤凰社总部。一个妖精跟在他的身边，战战兢兢地掏出一张羊皮纸，用沙哑的声音宣布誓言的内容。
如果邓布利多胜利，伏地魔必须放弃征服魔法界的野心，解散食死徒，接受威森加摩的制裁。
而如果伏地魔在决斗中胜利……
妖精的声音颤抖了。
伏地魔露出一个得意洋洋的微笑。那双燃烧的红眼睛里饱含恶意，令年轻的凤凰社成员毛骨悚然。赫敏在人群中悄悄握住了罗恩的手，不出意料地发现男孩的手心都是冷汗。
凤凰社沉默着，如同坟墓般死寂。
妖精的声音在屋子里回响。
如果伏地魔在决斗中胜利——
妖精说。
——他将迎娶哈利·詹姆斯·波特，巫师界的救世主！
“不！”赫敏听到自己发出一声尖叫。
她惊恐地在哈利和伏地魔之间来回扫视。年轻的救世主站在人群的最前面，高瘦的身影坚定地将凤凰社的其他人挡在身后。
她突然意识到，哈利已经长大了。
金妮在她身边发出一声窒息般的抽泣。
伏地魔扫了她一眼。
“年轻的格兰杰小姐和韦斯莱小姐似乎有话要说。”伏地魔轻声说。
食死徒队伍里激起一片笑声，或是为了迎合伏地魔语气中的嘲弄，或是为了嘲讽她们“肮脏”的血统。
赫敏的脸涨红了。
“你不能——”她鼓起勇气说。罗恩反握住了她的手。
她偏过头，迎上了男孩的视线。
罗恩看着她，眼里盛满了恐惧。
“赫敏，别。”
赫敏严厉地看着他。
贝拉特里克斯发出一阵尖利的笑声。
“小泥巴种的男朋友害怕了！”她尖叫道。食死徒哄然大笑，凤凰社的人脸色惨白。罗恩的脸涨得通红。他想要咆哮，却被赫敏给拉住了。莫莉·韦斯莱险些无法支撑自己的身体，亚瑟握着她的手臂，支撑着她。
“安静！”伏地魔的声音高亢而冰冷。食死徒的笑声立刻消失了，“交出波特，你们就可以离开了。”
“我们要查看誓言！”
米勒娃·麦格站在哈利身侧，头发一丝不苟地盘起。她坚定地看着他，摆出了一个保护的姿势。卢平站在哈利的另一侧，握住了哈利的肩膀。
伏地魔的眼睛眯了起来。
他挥了挥魔杖，没有理会凤凰社众人的畏缩。妖精手中的羊皮纸飘起来，悬浮在米勒娃面前。其他人凑到麦格身后，去看女巫身前的羊皮纸。
“请便。”他说。

罗恩挤在赫敏身边。
“这不是真的，”男孩小声说，“他在羞辱我们，对吗？”
赫敏脸色苍白。她用绝望的眼神看着他。
“不，罗恩。”
“赫敏，求你了——”
“妖精不会出错。”女孩低声说。罗恩注意到她的眼里盛满了泪水。“这是真的，罗恩，这个誓言是成立的……”
其他人一定也听到了她的话。莫莉呻吟一声，差点当场晕过去。金妮的眼泪落了下来，查理紧紧握着她的肩膀，让她把头靠在他的胸前。卢平的头低垂着，看起来非常痛苦。麦格的身体紧绷着，像是下一秒就要发射对着食死徒发送恶咒。
罗恩的喉咙发出了窒息的声音。
他忍不住伸着脖子去看哈利的反应。他的朋友站在队伍的前端，他毫无疑问看到了那张决定了他命运的羊皮纸。黑发的男孩脸色苍白，那双绿色的眼睛里燃烧着怒火。他的手紧握着魔杖，一动不动，仿佛变成了一尊雕像。
“我同意。”救世主的声音冷漠而坚决。
“哈利！”
“不！”
凤凰社的人大叫起来。罗恩发出一声沮丧的咆哮。他不知道……不知道该怎么办。为什么是哈利？为什么又是他？他为他的朋友感到痛苦。他拉着赫敏的手，让她不至于尖叫着向伏地魔扑过去。这一定是一场噩梦……
“如果我履行诺言，你就要放走凤凰社的人。”哈利盯着伏地魔，语气坚决，“你不能伤害他们，不能以战争中的罪行……追捕他们。他们会在你的统治下生活，就和其他没有反对过你的人一样。”
食死徒中爆发出一阵低语。伏地魔恶狠狠地瞪了他们一眼，那些黑袍人立刻沉默了。
“当然。”伏地魔说，“这也是誓言的一部分。”
莫莉发出了一声巨大的抽泣。伏地魔厌恶地看了她一眼，向黑发的男孩伸出手。
哈利看着那只手，克制住了自己后退的欲望。
他必须履行诺言。
他必须保护他的朋友。
他对着罗恩和赫敏宽慰地笑笑，但很可能只做了一个难看的鬼脸。因为他们的看上去马上就要晕过去了。
他只好苦笑着转过头，一个人直面伏地魔。
他能听到金妮的哭泣，能听到韦斯莱夫人小声地叫着“哈利”。他能感到大家的视线紧紧地黏在他的身上，刺得他皮肤发疼。一些食死徒不怀好意地打量他，好像在看一只陷阱里的猎物。
而他只是看着伏地魔。
真奇怪 。那个蛇一样的男人只是静静地看着他，眼里没有他想象中会出现的羞辱和不屑。他当然是想羞辱他……还有什么比伏地魔迎娶救世主更能羞辱反叛军的呢？
他咬着牙，握住了伏地魔的手。
瞬间，一股熟悉的扭曲感传来。他像是被塞进了一条旋转的橡皮管里。空气



✨  
**Topic Probability**: 0.4352547366421082  
**Document**: 下划线为原著摘取内容

在哈利握住那个高脚杯的瞬间，胃部翻滚着被拉扯的感觉让他恶心欲呕。

 

从昏沉中转醒后，他环顾了一下四周，显然，这里不是霍格沃茨，看起来也不像是什么颁发三强争霸赛冠军荣誉的特别领奖台。

 

好吧，看来那个奖杯被做成了一个门钥匙，他的生活还真是永远离不开意外惊喜。哈利默默地自嘲道。命运女神从来都不会放过任何一个捉弄他的机会，他早就明白这一点了。

 

男孩恼火又无奈地轻叹了口气。试图让自己酸痛而狼狈的身躯从地上爬起来。

 

不管这状况到底是怎么一回事，他这个第四学年过得是真的是很累了——虽然他的前三个学年也没有好到哪里去吧。此时此刻他只想好好歇一歇，洗个澡，闭上眼睛睡个觉，让什么魔法石密室三强争霸赛的麻烦事全都见鬼去。

 

当然，现实会敲醒他的妄念。只见这时候一个人影从黑暗中出现，朝哈利的方向走来，他看到那个家伙小手指断了一截，身材矮胖，然后，那个人——小矮星彼得，将自己紧紧地捆在了一个墓碑上。

 

而墓碑上刻着的名字是汤姆·里德尔。

 

这又是个什么情况？虫尾巴，和伏地魔的......不对，伏地魔的父亲的墓地？

 

哈利不得不认清这样一个事实：自己被迫参与的三强争霸赛的确是个彻头彻尾的阴谋，是个把它从学校带到他“命定的对头”——伏地魔的面前的圈套。

 

还有什么能比这更糟糕的呢？

 

沮丧的情绪顿时席卷了他的全身。但是哈利发现自己的反应其实比想象中要更冷静一些。

 

毕竟他一直以来就是这样活着的。甚至可以说是有些习惯了，为了一个所谓的属于自己的“使命”，作为“被选中的人”，他总是不得不和自己水深火热的生活抗争。如果说在他的第四学年里，就这样顺顺利利的拿到了比赛的冠军，然后淹没在美好地鲜花和掌声之中，那他一定会认为自己是在白日做梦。

 

男孩面无表情地垂着头，甚至懒得去试图挣扎一下捆缚在身上的绳子。就这样眼睁睁的看着虫尾巴划开自己的手臂取走几滴血液——说真的？他原本还以为对方是打算让自己慢慢失血而死呢。

 

不论如何，今天肯定就是他的忌日了，他默默的想。奇怪的是，这个念头并没有让他感到特别恐惧或者是恼火。

 

啊，是啊。作为一个14岁的青少年，从他出生没多久，还是一个婴儿的时候开始，就注定了自己要承受一般人无法承受的一切。

 

失去父母，被虐待自己的麻瓜亲戚带大，以为来到霍格沃茨能开启自己的新生活，但每一年的万圣节却都告诉自己，不，这只是另外一个开始。

 

他被追着赶着生活，瘦小的身躯扛着他难以扛下的重任——打败黑魔王，打败那个几个世纪以来史上最强大的黑巫师。讲道理，对方难道不会因为自己的“命定对手”是一个这样乳臭未干的小孩子而感到恼怒吗？

 

想到这儿，他不由得笑出了声，让一边蜷缩着身子的虫尾巴吓了一跳，睁大眼睛看着他。

 

 

随着念咒的结束，仪式也终于完成了，坩埚里缓缓走出来了一个身影，四周的强大魔力和虫尾巴的紧张与恭敬彰示了这个人的身份。黑魔王有了看起来比之前更具有人形的躯体，空气中弥漫着肃杀的氛围，哈利默默的闭上了眼睛。

 

今晚就会结束了吧。

 

荒谬的是，他居然感到一种解脱。

 

——他终于可以从这该死的、没完没了的生活里解脱了。

 

 


“哈利·波特，你正站在我父亲的尸骨上。”那个人轻轻地嘶声说，“他是一个麻瓜加笨蛋……就像你的母亲一样。但他们都有用处，是不是？你小的时候，你妈妈为保护你而死……我杀死了我父亲，你看，他死后派上了多大用场……”



 



伏地魔又笑起来。他一面来回踱步，一面扫视着四周。



 



“看到山坡上那所房子了吗，波特？我父亲在那里住过。我母亲是个巫师，住在这个村子里，爱上了他。可当她说出自己的身分之后，他抛弃了她……我父亲他不喜欢魔法……”



 



“他离开了她，回到他的麻瓜父母身边，那时我还没有出生，波特。我母亲生我的时候难产死了，我在麻瓜孤儿院长大……但我发誓要找到他……我向他报了仇，那个给我取了跟他同样名字的人……汤姆·里德尔……”


 

“我知道。”哈利小声地说。

 

伏地魔的发言被一直默不作声的少年突然打断，他似乎愣了一下，然后立刻怒气冲冲地问道，“你知道？你知道汤姆·里德尔这个名字？那个老傻瓜告诉你的？”

 

哈利叹了一口气。

他有些莫名地烦躁。

 

黑魔王原来这么啰嗦的吗？

就不能直接给他一个了断完事吗？

 

“这不重要，说真的，我不在意。”哈利的声音里听不出畏惧或者害怕之类的情绪，甚至淡定得有些冷漠，似乎还夹杂了一丝丝的不耐烦，这让伏地魔皱起了眉头。“我什么也不想说，也不想听你唠叨你的过去和现在，反正你不是要杀了我吗？就不能给个痛快？”

 

好吧，这一次伏地魔是真的愣住了。很快地他回过了神，眯起眼睛。

“哈利·



✨  
**Topic Probability**: 0.4321608577041078  
**Document**: 楔子  IF……

 

“说实话，直到现在，我都觉得，挺不可思议的。”赤褐发色的男孩微微偏头，轻声说道。

拥有华美金红色羽毛的鸟儿轻轻鸣叫了一声。

“可是，‘我’的记忆并不完整，最起码我现在看不到所有的。”男孩有些苦恼地叹了口气，“就算是我看过的，也没有什么感同身受的感觉。除了让我头痛以外，没有别的用处。”

这一次凤凰直接跳到他肩膀上，试图去啄他的耳朵。

“唔，我当然知道你不会弄错。”男孩连忙安抚性地用手指为她梳理羽毛，“只不过，任何人突然发现自己已经活了一个多世纪，总会有些难以置信的。这就好像你在毫不知情的情况下同时成为彩票中奖人和通缉犯，不管好事还是坏事，本来都和你没关系。”

这个拙劣比喻的后果就是凤凰非常不客气地啄了他一下，实打实的。

“好吧。”揉了揉红肿的耳朵，男孩连忙道歉，“我只是开玩笑而已。”

他从口袋里掏出了一封用厚重的羊皮纸做信封、绿色墨水书写且没有贴邮票的信。信封上有一个他不知道该说熟悉还是陌生的纹章。他轻轻摩挲着信纸，同时仔细审视上面的字迹:

伦敦东区
莎莉孤儿院
三楼西走廊尽头房间
阿不思·邓布利多先生收

这封信有很多不同寻常的地方，比如说它是被一只猫头鹰大清早飞进来扔到他身上的（孤儿院的邮箱前几天被撞坏了）；比如说信封是一种相当古老的材质；比如说上面的地址精确得不太自然。总之，这封信特殊到让阿不思在看见它的瞬间就将它一把塞进了口袋，以防可能发生的任何麻烦。直到所有人都开始午休，他才溜到天台来好好研究这封信。

 

阿不思所说的“记忆”，解封于半年前。后来他估计那本来该是一次小巫师的魔力暴动，放到他身上就打开了记忆的闸门，险些一发不可收拾。外在表现是他直接晕倒在了麻瓜小学的课堂上，并且高烧不退。

这是一场纯粹的意志上的战争，他必须坚守自己的神智，才不会被洪流一般的记忆冲击到崩溃。最终他赢了，那些记忆流动逐渐平缓，有的偃旗息鼓蛰伏不动，有的则缓缓流过灵魂视角的眼前——

他看见自己在辽阔美丽的沃土原上奔跑，看见自己在神秘幽暗的古堡中穿行，看见一场又一场战斗，看见从大门走进礼堂的一批又一批孩子……还有镜中眼神疲惫的自己，浓重的悲哀被压抑在眼底，只有他自己才能看出其中有多么深重的痛苦。

但他不懂。

那些感情不属于现在仅有十一岁的阿不思·**邓布利多**。

但他也无法忽视记忆对他的影响。所有义工都认为他比以前要沉静得多，仿佛一觉醒来就褪去了大半的稚嫩。他开始感到恐慌，开始质疑自己存在的真实性，尽管这些都没有被别人看出来，仿佛他生来就擅长保守秘密。

还有福克斯。凤凰奇迹般地找到了他，至少让他确定了自己的记忆不是臆想。

无论如何，正事都是要办的，更要珍惜宝贵的自由时间。于是一人一凤凰凑在一块，开始看信件的正文:

霍格沃茨魔法学校
校长：米勒娃•麦格
(国际变形术协会会长、巫师协会会长、梅林爵士团一级魔法师)

亲爱的邓布利多先生：
我们愉快地通知您，您已获准在霍格沃茨魔法学校就读。随信附上所需书籍及装备一览表。
学期定于九月一日开始。我们将在七月三十一日前对您进行必要的指引和解惑，敬请等待。
副校长（女）
波莫娜•斯普劳特  谨上

阿不思轻轻舒了口气，也不知是为没有“另一个自己”感到宽慰还是忧虑。如果其他人没有变化的话，那么霍格沃茨的情况应当还不错（他对两位“曾经”同僚的品性和能力都相当信任）。他更担忧的是黑魔王的情况。

1991年，真是一个再碰巧不过的年份。

是否还会有大难不死的男孩？

他需要做的，是根据现实作出判断。
 



✨  
**Topic Probability**: 0.3929127769413118  
**Document**: Chapter 1

哈利·波特的这十几年无非就是一场与黑魔王猫捉老鼠的游戏，他始终在向前奔走，在各路阴谋中逃亡，就连拿起魔杖迎接敌人时都从不认为这是胜利的前夕。
因此在他几乎无法动弹的被钳制在荒凉阴森的墓地时，释然反而比恐惧来的更快。
臂弯的伤口依旧在流血，伤痛在催促他清醒，可那盛装血液的小瓶彻底倒入沸腾的坩埚时，伤疤瞬间传来的触电般的窒息吞没了他，紧接着便是难以忍耐的巨痛。
头仿佛要炸开一般，湿粘汗水顺着额角滑落模糊了男孩的视野。他拼命眨着眼压抑呼喊，他看到了无法以简单的恐惧来形容的场面——他的死敌，在扭曲中重新拥有了身体。
伤疤传来的难忍的窒息感在伏地魔披上黑袍时得以缓解，哈利艰难的喘着气，他已经放弃挣扎出去的想法..除非眼前那个背对自己的巫师肯注意他哪怕一眼，不然凭他一己之力简直是妄想。
然而直到黑魔王召集食死徒，挨个惩罚完再把他们全部轰走，场地上只有他们两个人时，伏地魔才把目光彻彻底底的转向被束在死神雕像上的哈利。
如此残忍的忽视。
伏地魔似乎根本没把哈利放在眼中，这是男孩心中冒出的第一个念头。
以黑魔王的做事风格，当着所有食死徒的面把他狠狠折磨一顿再杀死才是正确的，这难道不是最有效的报复吗？他越是表现出不在意，哈利心中的不安便会多一分。
但伏地魔就是那样毫不在乎，他慢条斯理的迈着步子朝着男孩的方向靠近，白色的魔杖在他修长的骨节分明的手中捏握，哈利没有躲避，直直的抬起头注视他。
魔法重塑的身体让伏地魔几乎不可能有毛发的生长，他的面孔依稀可以看出往日的轮廓，但非自然的重生让他的样子无比惊悚..哈利压抑着几乎与愤怒持平的恐慌，望向伏地魔猩红的蛇瞳中。
“你比我想象中的要安静..哈利·波特..”伏地魔带着些打量开口了，声音沙哑到让哈利头皮发麻，“把你弄过来浪费的心思远比我想象中的要复杂..不过没关系，你还是准时的出现在这里..”
“我很高兴你能赴约..男孩儿，所以你可以赢得黑魔王的部分奖赏..”
伏地魔还在靠近，他的声音也越来越低沉，伴着刻意的嘶嘶声疯狂的钻入哈利的耳朵中。白色的紫衫木魔杖受苍白指尖的控制抵在他的腰腹上，灵巧的绕出弧线触上左胸口的位置，那里是男孩因压抑而剧烈跳动的心脏。
“有没有奇怪?三年前我甚至都没办法靠近你....”哈利在魔杖游走在身体时细微的颤栗，部分来自对未知命运的恐慌，部分属于身体被触碰，剩下的则是伏地魔几乎紧贴他耳侧脱口而出的话语。
他说的没错，一年级时奇洛根本承受不住自己的触摸。哈利莹绿的眸子微微颤了颤，受伤的手臂不安的扭动着，因为他看到伏地魔那纤瘦的手指伸了过来，力道不算轻的捏了上去。
“嘶..”他吸了口气，但很快就重新咬住了声音。
“真是可惜..如果不是为了今天，我十分不希望看到你流血..哈利..”伏地魔那故意放缓放轻的嘶嘶声听的哈利后脊发凉，更不用说他话里话外的意思了，不想看到他流血?简直是放屁。
伏地魔巴不得他赶紧去死，哈利死死的盯着转移到自己手臂伤口上的魔杖，心又开始扑通扑通的跳。
但实际上什么恶咒都没有出现，反倒是有一束微弱的光投了出来，狰狞的刀伤停止了滴血，逐渐开始愈合。
哈利楞在那里，但他很快就想起虫尾巴的那个丑陋又充满危险的假手，顿时厌恶的皱起眉。
“很简单的治愈咒，你在乱动什么？”伏地魔似乎有些不满，指尖触上哈利扭向一边的下巴强行让他正视自己，原本腥红的蛇瞳更加可怖。太近了...他甚至注意到伏地魔刚刚重生的身体仅有一件作为遮挡的黑色袍子，这导致他稍一低头就能看到黑魔王外露的清晰锁骨，与若隐若现的胸疼。
不知为何，伏地魔似乎十分享受碰触他的身体。
哈利被他的力道捏的生疼，但臂弯的刀伤的确已经愈合，可伏地魔想让他做什么？说出感谢的话吗？哈利的表情更加厌恶，他直视着对方的眼睛，原本几乎悬空的双腿奋力抬起就想把他踹开。
伏地魔明显没料到哈利波特会用这种完全倾向麻瓜行为的反击方式，但他还没弱到真的会被偷袭的程度，几乎是在男孩抬脚的瞬间，一股无形的力量便扯住他的腿弯，强硬的朝外扭到一侧。
“在霍格沃茨四年反倒是把你教成没教养的小鬼了吗？”黑魔王有些恼怒，他那如骸骨一样的手掌按在哈利大敞的腿部内侧，引得后者浑身抖了一下。
伏地魔或许真的不打算给哈利造成伤害，这完全是一个自小就被标记为死敌的男孩意想不到的，但他也的确在被拘束，他不得不蜷起另一条腿与被伏地魔按住的那条尝试并拢来缓解有些尴尬的姿势，梅林，伏地魔没把他腿扭断已经是万幸了，不然他宁可死在这里也要和这个该死的蛇脸同归于尽。
“只要你选择服从，我自然不会委屈你今后的生活..”伏地魔依然在引诱式的开口，“霍格沃茨也不会被渗透。”
“你是个骗子，汤姆...”哈利成功的看到黑魔王因自己的称呼而不耐的皱起眉，他从没真正的恐惧过什么，因为那些和他经历过的相比全部都不值一提，“你学生时期就在欺骗所有人，你现在也在欺骗我...我凭什么相信你的话？”



# Sex?

In [41]:
get_top_docs(original_texts, topic_distributions, topic_index=14, n=5)

### ✨Topic 14✨

['信息素', '生殖腔', '发情期', '避孕药', '抑制剂', '灰蓝色', '斯科皮', '青苹果', '百分百', '阿尔特弥斯', '不由得', '腺体处', '哈利则', '面红耳赤', '生理性', '危机感', '哈利压', '哈利点头', 'beta', '压迫感']

---

✨  
**Topic Probability**: 0.304797568868692  
**Document**: 他是一点一点被吃掉的。先是乳头，被含在嘴里咂，被叼着磨。

“长大了一点。”幼小的乳房，白白嫩嫩鼓成一个小馒头，哈利声音哑得有些古怪，像是被嘴里的胸乳甜齁了嗓子。那两颗小尖尖被又软又湿的舌头滑过，被尖牙咬着，在哈利的口腔里颤抖。他放低声音求他：“别咬……要扯掉了……”

他的身体吃下两根手指、一根舌头，哈利吃下他被催熟的乳肉和湿淋淋的快乐。他两腿间的花被舔开，薄薄的两瓣被粗鲁的分开，藏在深处的粉色嫩肉暴露在空气里，哈利在他的腿间喘着粗气，像烈日里晒坏了的狗，伸长舌头吐着热气。他小小的阴部被打湿了，舌头，舌头是软的，烫的，像条灵活的蛇钻进他的身体里。他的穴肉着了火，又痒又酸直流汗。哈利像吸果冻一样吸他，先是浅浅地嘬几口表面的汁，再重重吸去最后一口。那一口直把他的灵魂也给吸走了，他抽筋似的乱蹬着两条光溜溜的腿，爽得呜呜直哭：“不要了，好舒服，呜，不要……” 

那根舌头在里面杀不了痒，戳得他愈来愈难受。咕叽咕叽的水声从他的身体里传出来，他变成了一条春潮中的小溪，那些乱荡的水在他的哭声中从舌头与穴口的缝隙中流出来，浸湿一小块墨绿色的床单。

哈利凶狠地瞪他闭不拢的女穴，像是恨它弄脏了床。他神情可怕，在床上吓唬他：“我要吃了你。”他看起来那么认真，眼睛里是恶狠了的一片青，盯着他的身体，舔着干燥的嘴唇，张嘴吃他身体上最软的肉。

“不要！”他哭着推他的头，他下面的小嘴一缩一缩地还没缓过来，哈利就又饿了。

“让我舔舔，你会舒服的。”哈利强硬地按住他的手腕，整张脸凑到了他的两腿间，炙热的呼吸喷在他的穴口，他的穴好小，被一口含住的时候吓得发抖。“不要，唔，不舒服……”他叉着腿扭着屁股想逃，被吸得又哭又叫，鼻涕也流了出来。那张嘴包着他又嫩又软的肉，舌头拼命往他身体里钻，嘬着他外露的阴唇，把他下体弄得又湿又痒。他一个劲儿地哭，身子一颤一颤，身下的穴也一下一下地夹着那根舌头。平时帮他擦去眼泪的人手正牢牢捏着他的手腕，张着嘴含他的穴。他热得直流汗，屁股扭来扭去想从哈利的嘴里逃出来，那根舌头却狡猾地顺着他的动作往他身体里滑。他本来是平躺着的，为了躲开那张烫人的嘴，腿越抬越高，最后抬不动了只能搭在哈利的肩上，只方便了人来吃他的穴。

“别吃我了，好不好，哈利……”他被自己的口水呛到，眼泪糊住了眼睛，朦朦胧胧看见哈利抬着他的屁股吃他的肉。哈利吃得他好难受，他被吸得又胀又痛，下面麻麻的合不拢。他被舔坏了，那里原本是紧闭着只看得见两片软乎乎的阴唇，现在被欺负得露出了粉嫩嫩的肉。

“多舔舔才会长大，小龙，不舒服吗，你流的水都把我喂饱了。”哈利停下动作将他抱在怀里，凑到他耳边笑着哄他，声音却是装出来隔着层古怪的温柔。德拉科哭得直抽气，他伸手摸了摸自己湿漉漉的阴部，想不通为什么自己流出了那么多水喂饱了这个坏蛋。

第二个被吃掉的是舌头。哈利把他推到角落里：“舒服吗？和你下面的小嘴学的。”

他讨厌哈利说这种自作聪明的话，难道哈利以为他不知道他做了什么坏事吗？他早就知道，下面的小嘴是不能给别人舔的，乳头也是不能给别人吃的，他全身上下都和哈利的嘴没关系。可是哈利瞒过所有人舔遍他的全身最后还骗他接吻。

牛轧糖的味道让他并不抵触另一个人的舌头，但是哈利却从没有意义的唾液交换中得到了乐趣。硬邦邦的阴茎隔着睡裤抵在他的小腹上，哈利抓住他的屁股让他贴着那根阴茎。他在他身上蹭，像到了发情期的野兽，嘴里不干不净地说着荤话：“湿了吗，想不想被舔”

在他能够回答之前，一只手已经伸到了他的裤裆，狠狠揉了一把他的下体。

“内裤湿了吗？我摸摸。”

他夹紧了腿，他本来没有湿的。

腿缝里被强硬地塞进手掌，两根手指快速扣着他凸出来的肉。那两瓣肉闭合在一起，被扣的又酸又麻。他站不住，趴在了哈利怀里。手掌隔着裤子按在他的穴上拼命地磨，像是故意要把他弄坏，他害怕得踮起了脚又跌回手掌上，哆哆嗦嗦被送上了高潮。

“你弄坏我了，别扣了，我不和你玩儿了……”他说不顺一句话。他本不是嘴笨的人，他聪明到擅长一个学生应该擅长的所有事情，他能不说一句废话写满五页牛皮纸的论文，能轻而易举把箱子变大成一张床，却不知道在情欲里应该怎么求饶。

哈利又埋下头去亲他，含着他的下嘴唇舔他下排的牙齿。他看过他父母接吻，柔软的嘴唇贴在一起，像两片被风吹在一起的玫瑰花瓣。而哈利吻他却像是要将他吞进腹里，吸卷他的舌头像是要钻进他的胃里。哈利看他的眼神像在看自己深夜十二点的晚饭，他看他总像是饿极了的样子。

“别这样了。”他使劲推开哈利，被舔破的嘴唇火辣辣的疼。他全身是汗，心里却慌得打颤。

“怎么了。”哈利直勾勾地盯着他破掉的嘴唇，像是还想啃上一口。

“我知道你在干什么。”他想他知道这是怎么一回事，从最开始这一切都不对。接吻是恋人间的事，所有的一切都是恋人间的事情，他们只是太早闯进了成人的世界，他们被迷晕了头，懵懵懂懂跳进



✨  
**Topic Probability**: 0.21161590546768602  
**Document**: “Okay, you guys are free to go."  　　教魔咒课的哥布林教授挥了挥小短手，示意学生们离开。他踩着用来垫高的箱子一点一点下了讲台，学生们也纷纷站起来收好自己的魔杖和书本，有说有笑地出了教室。　　      “Please wait,professor."等教室里的人快走完了，哈利把魔杖收进袍子口袋，飞快地弯腰用一只手抱起书，叫住了马上要走出教室的教授。  　　“啊，是Mr.Potter, 有什么事吗?”教授转过身，和蔼地问道。　　      “um...我只是...”哈利涨红了脸，“我最近可能不是很方便来上课...所以我想——”  　　“请假？”哥布林教授扶了扶眼镜，了然地点点头，他知道人类Omega每隔一段时间总会有一次热潮期需要呆在家里度过，况且哈利是他最优秀的学生之一，即使落下课程也没有什么大问题。所以他只是微笑着拍拍哈利的手臂说：“好，我知道了。”　　      哈利点点头表示感谢，轻轻舒了一口气。  　　今天他已经感觉到体温升高浑身无力，他知道这是发情期要来之前的先兆，最迟明天，真正的发情期就要到了。不过对于一个有了Alpha的Omega来说，发情期无非也就是费事一点，缺几节课而已，而且有赫敏在，缺课简直都不叫事儿。　　    男孩揉了揉发烫的脸颊，刻意拉起袍子上的黑色兜帽仔细戴好。后颈有很淡很微弱的奶香信息素在往外散逸，他能闻得到。虽然不至于引诱到其他Alpha，他也不想在外逗留太久。  　　德拉科此时应该已经回到房间等他，快发情的Omega躁动不安的情绪需要德拉科的信息素来安抚。　　     哈利想着，有些焦急地推开了教室的门。  　　开门的瞬间，他脑中突然有一瞬的晕眩，耳朵嗡鸣起来，使他有些站立不稳地踉跄了一步。袍子下露出的纤白的手指连忙扶上门，头抵住门框，他皱着眉闭眼缓了一会儿，再睁眼时，晕眩感已经消失了。他只当是发情期前的不适反应，也没在意。　　      为了避免有人注意到他，哈利一只手抱着书，另一只手拉住兜帽，低着头走的十分快，但路过的学生还是有人忍不住悄悄看他。这也很正常。哈利是个过分纤细好看的Omega，祖母绿的眼睛像一汪春水，盯着你看的时候能把你的灵魂看化了——这是德拉科说的。哈利在还没分化时就有许多人追求，他们甚至不介意他最后会分化出什么性别也想拥有他，德拉科为了这个不知道吃了多少醋。于是在哈利第一次发情的时候，德拉科就标记了他，彻彻底底地把他变成了“Harry·Malfoy”。不过还没有彻底成结，因为在Omega生殖腔内射的受孕率实在高得可怕，德拉科不舍得让他的宝贝这么快就怀孕。  　　　　哈利歪头站在宿舍门前，迟疑了一下。  　　刚刚来的路上他总感觉有什么不对劲，虽然他也说不上来，但能肯定的是整个霍格沃茨确实弥漫着一种十分反常的氛围，让他感觉浑身不舒服。　　       可能只是自己太敏感了？  　　哈利使劲闭了闭眼，尽力让自己忘掉这种不寻常的想法。他知道发情期前的Omega总是敏感又脆弱，容易多想。　　况且，他有些焦躁地舔了舔干涩的嘴唇——  　　现在也的确不是时候。　　摇了摇头，哈利推开门，一下子撞上刚从浴室走出来的德拉科震惊的眼神。  　　“波...波特？”德拉科皱着眉，摘掉自己漂亮的铂金头发上的毛巾，不知是想起了什么，情不自禁地后退了一步，“你怎么进来的？”

 

        屋子里充斥着肆无忌惮蔓延的Alpha信息素，壁炉里的火焰炽烈，炙热的青苹果香气浓郁且强势地冲击着哈利的感官。平时因为哈利太过敏感，德拉科通常不会这样毫无顾忌地释放自己的信息素，但是今天却一反常态。

      哈利看着穿着浴袍的小少爷精致至极的雪白面容，铂金色的发丝垂在脸侧，还在往下滴水。他眯起眼睛，强烈地感到自己几乎要溺毙在他的Alpha摄人心魄的美色里，浑身都酥软下来，无力抱住的魔法书掉落在地上，兜帽也滑落下来。
 
 
　　书本摔落的响声让德拉科回过神，他动了动鼻子，敏锐地捕捉到空气中渐渐散发出的一丝香甜的奶味，震惊到连手里的毛巾都掉了。
　　
“你..你快发情了？”德拉科皱着眉，用一副看怪物似的表情，看着哈利泛上艳丽酡红的脸蛋，“没搞错吧，波特，你一个快发情的Omega，跑到我房间来了？”
 
 
　　哈利迷蒙又疑惑地抬眼，还没来得及探寻为什么今天德拉科的态度如此不同，下巴就被抬起，对方轻柔又炽热的呼吸像羽毛一样轻轻搔着他的脸颊，Alpha霸道的信息素充斥鼻端，让他有些羞于看德拉科灰蓝色的眼睛，只是盯着地面，脚尖划着地毯，身体又开始发烫。
　　

      “哈...”德拉科冷笑了一声，盯住哈利被泪水浸泡着的绿眼睛，一只手绕过他的脖颈，撩起后脑柔软的黑发，发现那飘散着奶香的小巧腺体上居然没有被标记过的牙印。更诡异的是，这个一直对他冷冰冰的救世主不仅没有对他的行为表



✨  
**Topic Probability**: 0.19570875258233358  
**Document**: “什么！？结婚？！马尔福你疯了！”

 

“别这么激动，疤……波特先生，这对我们彼此都有好处，不是吗？”

 

一次出乎哈利预料的谈话，这是一切的起因。

 

预言家日报-头版头条：

 

《救世主“哈利波特”订婚！订婚对象竟然是！？》

 

几个月后，一场盛大的婚礼举办得轰轰烈烈，话题屠版各大新闻媒体，魔法界几乎没人不再谈论着救世主结婚的事情。

 

格里莫广场12号。

 

哈利浑身不自在的坐在沙发上，他的对面是拿着杯红茶喝的慢条斯理的马尔福。

 

此时的房子已经焕然一新，虽然他一直都有打扫，但是这么大的房子他能用到的地方其实并不多，好多房间都还保持原样搁置着。

 

而马尔福的到来显然更能激发克利切的激情。

 

看啊，现在的房子就像是一个真正的斯莱特林纯血该住的地方，又或者说这才是这间房子本来该有的样子。

 

他自己最喜欢的格兰芬多金红已经被另一种颜色强势入侵。

 

但是哈利现在还顾不上这些，他看着对面的马尔福，依然觉得发生的这一切都太过于荒谬了。

 

他竟然真的和马尔福结婚了！

 

就算……

 

就算他们已经不再像以前在学校里时一样，一见面就必是针锋相对剑拔弩张，但那也是个马尔福啊！

 

好吧，好吧，想开点哈利，就像马尔福说的，他们只是互相利用而已。

 

他自己需要在魔法部更近一步，马尔福需要他的“身份”转变他们家在战后的形象得到正面的支持。

 

马尔福都不觉得有什么，他在这里一直纠结个什么劲。

 

更何况马尔福是个beta，没必要担心什么。

 

哈利深呼吸一口气，也端起茶杯抿了口茶，润了润干燥的喉咙。

 

只是各取所需而已，不用想的太复杂。

 

但是该死的，就这么结婚不会太随意了吗？

 

起码他想象中的婚姻不是这个样子的。

 

德拉科不动声色的抬眼观察着坐立不安的哈利，无声的嗤笑了一下。

 

他不用想都知道波特纠结的点在哪里，还真是一直都没变。

 

不过就像他们一开始谈好的一样，这场婚姻只是双方互利的合作而已。

 

现在是重要时期，而他们斯莱特林出身的，从来都是为达目的不择手段的。

 

等一切都安排好，稳定好，这场只有利益的婚姻关系就会结束。

 

想到这，德拉科将茶杯放回到茶几上，抬眼又环视了一遍周围的摆设，勉强收起了自己嫌弃的表情。

 

“那么，相处愉快了，波特先生。”

 

哈利僵硬的点点头，然后全身都觉得不对劲的看着马尔福挑选了一间采光好还很大的房间。

 

直到人的身影消失在客厅后，他才向后瘫倒在了沙发上。

 

荒谬，太荒谬了。

 

和马尔福结婚后的日子并没有哈利想象的那么难以忍受，他也是深刻的体会到了斯莱特林留给对方隐私空间的细微程度。

 

可以说，他们虽然住在一个家里，但是几乎见不到面，甚至哈利觉得他见到马尔福的次数都没有以前在霍格沃茨的时候来得多。

 

然而见不到面是一回事，另外一个人有没有存在感就是另外一回事了！

 

作为一只狮子，哈利的领地意识其实是非常强的，但是现在他的领域内处处都充斥着另外一个人的痕迹。

 

就算不是物品，不是颜色，也有味道。

 

没错，就是味道！

 

以前在霍格沃茨的时候，哈利并没有注意到这个，可能他们能同处一室的时间和机会都不多。

 

但是马尔福住进来之后就不一样了。

 

整个房子里面，哈利总是能若隐若现的闻到一丝丝的甜味，那味道几乎无孔不入，他从没觉得自己会对一个味道这么敏感，而他甚至不得不承认，那味道非常的好闻！

 

房子里除了他就是马尔福，这个味道是谁的不言而喻。

 

但哈利搞不懂，马尔福身上的味道怎么飘的到处都是，他一个beta又没有信息素的味道，这又是马尔福家搞的什么奢侈的香料吗？

 

但该死的这个味道为什么他妈的见鬼的好闻！

 

一个好闻的味道当然不会让哈利这么苦恼，但那个味道对他的影响就完全不能够忽略了。

 

那是个让人尴尬的影响。

 

当某一天他们终于难得的在房子里见到面的时候，哈利没忍住对马尔福说道，“马尔福，你有考虑过换一种味道的香料用吗？”

 

“什么！？”

 

德拉科突兀地听到哈利的话后条件反射的向后退了一步，旁边柜子上的东西噼里啪啦的掉下来碎了一地，然而德拉科完全没去管那些，他立刻抬起手臂闻了闻，脸色在那一刻变得狐疑又难看。

 

哈利没想到马尔福的反应会那么大，最后等待他的是恶狠狠地瞪了他一眼后摔门离去的马尔福，而哈利至今都搞不懂他到底又哪里惹到他了？

 

只是一个香料的事，不想换他又不能逼他什么，他也就问问，至于吗？

 

果然他和马尔福没有愉快相处的可能。

 

等到马尔福再回来之后，哈利能感觉到那股味道变得比之前更淡了一些，但是实话实说，对他来讲和以前并没



✨  
**Topic Probability**: 0.156454837669025  
**Document**: 小王后
/曼达叽

（上）苦 Bitter
在Theseus Scamander王子六岁的时候，他的父亲南王Landors Scamander和弟弟北王Vinks Scamander结为了南北联盟。Vinks亲王许诺，让他的王后生下的第一位嫡子同王子成婚，并誓要将盟约延长，直到Scamander家族的后裔坐上王位。

-
Theseus彬彬有礼地对着北王的王后行了礼，对方温柔地亲吻了他的额头，笑着问：“殿下，要看看Newt吗？你的表弟。”这个聪慧而知礼的女人没有引申出婴儿未来的其他身份，说着，她将摇篮微微向他倾斜。
Theseus皱着眉头看着摇篮里那个普普通通的婴儿，他躺在摇篮里柔软的被絮中，饱满的脸颊红扑扑的，嘴角还带着点口水。人人都说这就是他未来的妻子，现在却还只是个流口水的小东西。看起来他离长大、成为一个王后还有很长的路要走。
于是他伸出手，轻轻地碰了碰那小小的手心，小婴儿睁开了眼睛，他完全遗传到了妈妈的眼睛，它们不是Scamander家标准的灰蓝色，而是褐绿色的。他的小手轻轻握住了Theseus的指尖，困惑地望着摇篮上方的Theseus，然后对Theseus柔软地笑了笑。

融化的蜜糖从他的心尖滴滴答答地落在他的心口。
北王后轻柔地问道：“喜欢吗？”
很喜欢，非常喜欢。Theseus这样想着，礼貌矜持地回答道：“小殿下很可爱。”

-
北国的小王子一天天长大，是的，就像几年前Theseus殿下说的那样，他很可爱，可看起来却不是个当王后的好料子。
没有哪个未来的王后会在和优雅的女老师鉴赏名画的时候，从衣兜里钻出来一只田鼠，把女老师吓得花容失色；在酒席上跟着一只闯入了大厅知更鸟转圈儿结果在城堡里迷路；更不会在花园里和小狗打盹儿一直睡到错过了晚餐。
当然，无论你问谁，Newt殿下怎么样，他们都会用“delightful”，“lovely”这些可爱的词语来形容他。当然咯，殿下确实配得上一切可爱的词语，他有着红棕色的卷发，褐绿色的眼睛，还有长着小小雀斑的面颊。
“孩子是会长大的。”人们这么说，慈爱地看着小殿下，他们相信他最终会成为一位优雅端庄的王后。

（中）甘 Sweet
-
可是七岁的Newt却并不想当王后，他比较想成为一个牧羊人——能在草原上放牧的那种，他曾经恳求过父王给他一头新生的小羊羔，却被拒绝了。
“你的鹦鹉打翻了你母亲的花瓶，”Vinks王严肃地说，“Newt，养些小型动物就可以了。要我说，你应该和你的表哥Theseus一样，放更多心思在政治、艺术、文学上。他像你这么大的时候已经可以绘制出完整的王国地图了。”
啊，又是Theseus表哥！Newt最终只能回到华丽的书房里，打着瞌睡上礼仪课。他更小的时候常常能见到Theseus表哥。六岁以后和Theseus表哥一年才能见一次，似乎所有人都在比较他们，认为他因为不懂军事和政治而不够格和Theseus并肩。
可是他却无法讨厌Theseus。

-
人们对南王未来的王后抱着些怀疑的态度，然而未来的南王，Theseus殿下，却已经狂热地爱上他未来的王后。
当然，十五岁的小王子是不会把这些汹涌的炽热爱意倾诉给一个小孩儿的，他还是和人们所期望的那样，矜贵、沉稳。

圣诞节的时候，北王的小殿下如约乘着马车来到了南国度过圣诞，临走前他的父母百般叮咛让他表现得体。
“Newt殿下，你过得好吗？”Theseus行了平礼，然后问道。他心中狂喜，目光不舍得在Newt脸上移动一点儿，生怕错过了他一个细节。一年不见，这孩子长高了点儿，穿着繁复的礼服，就像个活了的精美玩偶，十分惹人怜爱地望着他。
Newt很快垂下脑袋，行了平礼，回答道：“我很好。”便急急忙忙得想要跳下马车，然后逃开——他实在是不喜欢这样的场合，在花园里，或者和小伯爵Jacob在骑马的时候，他也是个活泼健谈的孩子。可是此刻Newt只想要逃到别人看不到他的地方去。
Theseus轻轻握住他的手，温柔地阻止了他钻到自己的安全角落去。

“Newt，”他耐心地说，注视着孩子的眼睛，他问：“你想看看你的圣诞礼物吗？”他引着Newt走到花园的橡树下，那儿卧着一只通体乌黑的小羊羔，Newt一下子扑上去，抚摸着小羊羔的软毛，仰起头问道：“他叫什么？”
“他是你的，你想叫他什么名字都可以。”Theseus说，“我听说北王陛下不给你养羊羔。我给他写了信，过完圣诞后，你可以带走他。”他心满意足地看到了Newt快乐的样子。
这真是最好的圣诞礼物了，他想。
然而他最好的礼物最终在晚宴结束的时候出现，在晚宴结束后，Newt才来主动和他搭话，他牵着Theseus的衣角，大厅里很喧闹，他的表兄半蹲下去听他说话。
“谢谢，我… …我很喜欢这个礼物，它叫Coco。”他说，然后鼓足勇气，亲吻了一下Theseus王子的脸颊。然后他抱着自己的羊羔，快乐地去找小女仆玩了。




✨  
**Topic Probability**: 0.14046848808294354  
**Document**: 哈利低头望着这些又好奇又紧张的孩子，其中一些还在窃窃私语的讲着流传了世代的被成年人默契隐瞒着的‘残酷恐怖的分院考验仪式’，真的从来就没有一个巫师家长对此说了真话，这是哈利这么多年来都无法理解的一件事。

 

数了数人数，他清清嗓子，“欢迎来到霍格沃茨，现在，跟着我。”

 

这应该是副校长的工作，哈利转身打开礼堂的门时一眼看见大厅另一端稳稳坐着的阴沉男人，这应该是那个人的工作。

 

他把那个人从死亡的边缘拉回来，对于那个人的清白他据理力争，比被审判的本人都在乎，而那个人做了什么回报这种倾囊相助呢？坚决拒绝校长一职，退居副校长的位置并继续教授黑魔法防御术，在他回到霍格沃茨任职之后十分不情愿的让斯拉格霍恩教授得以再次退休，重拾魔药教授的威严，并且对他十分不友好，不，是非常恶劣，还把本该是副校长的工作推给他，混蛋。

 

哈利领着新生走到分院帽前，转身的时候把脸上不甘的表情换成和善的微笑，打开名单，开始分院仪式。

 

无论那个男人如何不情愿，教师长桌的安排从来都是黑魔法防御教授在魔药教授的旁边，他坐在自己的座位上，习惯性的冲男人笑了笑，仍然没有得到任何回应。

 

这是他回霍格沃茨当教授的第二年，而他之前的一年里从那个男人嘴里得到的单词不多于五个。

 

“当然。”

 

“可以。”

 

“知道了。”

 

以及……

 

“让开。”

 

他敢发誓，在他三年级的时候，自己的位置上坐的是卢平，那个男人的态度都没有如此不可救药。

 

看来自己得到他记忆这件事儿真的是他宁愿死都不想面对的啊！

 

“你就那么记恨我抢走黑魔法防御术这门课吗？要知道斯拉格霍恩教授总归是要退休的，愿意跟一堆不定时爆炸的坩埚呆在一起的魔药大师真的不多了。”哈利在麦格校长宣布校规时悄悄的对他说。

 

“那你有没有想过我也不愿意呢？”斯内普的嘴角几乎没动，但语气中的嘲讽并没有因此减少一丁点。

 

他说话了！哈利一边在心里放了个烟花，一边对于自己的反应啼笑皆非，但他不能表现出来，就像你在野外遇到野兽，不能对于对方的任何反应大惊小怪，不然野兽会警惕的转身就跑的，“如果你还是校长就不会有这个问题了。”

 

这是个错误的回答，野兽的嘴唇薄薄的抿成一条线，脸上弥漫的嘲讽都被冰冻住了，“是啊，然后就永远都不会有人忘记我是怎么当上校长的了是不是？靠杀了前任校长？这样无疑很符合你的预期，不能让任何人包括我自己遗忘我的卑鄙和对比你的伟大，这是我什么需要付出的代价吗？”

 

那个回答真是错的不能再错了，哈利开始绝望的想要拥有一个时间转换器，用了一年多换来的一次交谈，这是最完美的搞砸它的方式。

 

“我希望人生中仅有一次，你能不曲解我，用善意的直白的意思来理解我的话，看起来我只能期待奇迹了。”哈利压抑着想要幼稚的用大喊大叫来争辩的冲动，他当了五年的傲罗和一年的教授，他不再是在对方办公室跟那个人学习大脑封闭术的青春期少年了，绝对，绝对，绝对不能让自己失控。

 

“让我们都容易一些，别试着交流，你觉得怎么样？我觉得可以接受。”

 

“很遗憾，我觉得不行。”麦格校长已经宣布开始晚宴了，食物出现在餐桌上，哈利带着被挑起的斗志把几块玉米盛到自己的盘子里，他在去年学会的一件事，不能在生气的时候吃太油腻的东西，那样会让自己生气的同时整夜胃痛。

 

这个结论完全就是斯内普替他总结的。

 

斯内普没有再说话，哈利很高兴的注意到他的食欲因为自己也所剩无几，两败俱伤好过他一个人闷闷不乐。

 

跟斯内普打交道要学会知足，可是要狩猎野兽却恰恰相反。

 

哈利冲着盘子里的玉米绽开一个微笑。

 

 

斯内普知道自己不适合当一个教授，因为他总是不由自主的把这群战战兢兢看着他的学生跟同龄时的自己对比，然后他就更为不能容忍这些小鬼的愚不可及。

 

他一直认为魔药是复杂而优美的学问，想要接近她首先要不是一个只会挥魔杖的蠢货，所以他才更想教黑魔法防御术，最起码在那门课里他不会总是因为魔药学本身被冒犯了而感觉自己快要心脏病发。

 

例如现在，一个不知道往自己的解毒剂里加了什么他完全无法理解为什么会出现在这个学生脑海里的东西导致他的魔药变成了某种催化剂，这个六年级的班级里有一个倒霉的刚刚过完十七岁生日的成年学生，或者说，刚刚分化的Alpha，被这种催化剂诱发了信息素，那个学生是怎么进入他的高级魔药班的？

 

现在不是回想自己的疏忽的时候，他咬着牙当作什么也没察觉，把弄砸魔药的学生扣了五十分，理由是‘出现在他不该出现的与自身能力不匹配的教室里’，然后继续上课，对，必须继续上课，因为这个教室里不该有人察觉一个被诱发信息素的Alpha，他只能希望那个被诱发的蠢货自己提出不适要申请去医疗室，他的所有希望都寄托于此，但是没有，他已经有一个愚蠢



# Old Wizards?

In [ ]:
get_top_docs(original_texts, topic_distributions, topic_index=22, n=5)